# Simulated Annealing (Имитирано закаляване)

<div class="simulated-annealing-algorithm">
    <img src="images/simulated-annealing-algorithm.png" alt="Simulated Annealing" style="width: 700px; margin: 10px;"/>
</div>

### Характеристики:
- вдъхновен от металургията - залаляване на метал
- итеративен алгоритъм
- евристичен (няма доказана сходимост)
- стохастичен (използване на произволни числа)
- MCMC метод (Markov Chain Monte Carlo)

### Параметри:
- **T, Tmin, Tmax** :: температурата е основен параметър, който определя поведението на алгоритъма
    - в началото, когато T е голямо, алгоритъмът прави разходка в ширина (random walk, )
    - към края, когато T е малко, алгоритъмът експлоатира локален екстремум (hill climbing, обхождане в дълбочина)
- **INIT()** - начална точка / начално решение.
- **NEIGHBOUR(T, best)** - дава следваща точка в зависимост от температурата и настоящата най-добра. Например точка от нормалното разпределение около точка **best** с дисперсия **Т**.
<div class="normal">
    <img src="images/normal.png" alt="Normal Distribution" style="width: 400px;"/>
</div>

- **ACCEPT(T, dE)** - определя с каква вероятност ще приемем по-лоша точка.
$$\displaystyle ACCEPT(T, \Delta E) = \mathrm{e}^{\frac{-\Delta E}{kT}}$$
- **ENERGY(X)** - целевата функция, която се оптимизира. При дискретните проблеми трябва да си измислим такава.
- **COOLING(T)** - правилото, по което се намалява температурата **T**
$$COOLING(T) = \alpha T \hspace{2pc} \alpha = 0.99$$

### Поведение:

- Explore $\displaystyle T \to \infty \hspace{1pc} \lim_{T \to \infty} \frac{-\Delta E}{kT} = 0 \hspace{2pc} \mathrm{e}^{0} = 1$

- Exploit $\displaystyle T \to 0 \hspace{1pc} \lim_{T \to 0} \frac{-\Delta E}{kT} = -\infty \hspace{2pc} \mathrm{e}^{-\infty} \to 0$

### Плюсове:
- лесно адаптиране на алгоритъма към даден проблем (непрекъснат или дискретен)
- не налага никакви ограничения върху целевата функция **ENERGY(X)**
    - няма нужда **ENERGY(X)** да е непрекъсната, може и да е **дискретна**
    - не изисква например да е гладка (да има N непрекъснати производни), производната да е Липшицова или Хесиана да е положително полу-дефинитна матрица, както при градиентните методи
    - няма ограничение за изпъкналост (единствен оптимум)
- паралелизъм / последователно пускане на алгоритъма там, където е приключил


### Минуси:
- евристичен, тоест не е гарантирано, че ще намерим оптимум
- не знаем колко близо сме до оптимума


In [36]:
import operator
import numpy as np
import math
from numpy import random

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import matplotlib.animation as animation

%matplotlib nbagg

class SimulatedAnnealing():
    def __init__(self, minimize, T_min, T_max, init,
                 neighbour, energy, accept, cooling):
        self.compare = operator.lt if minimize else operator.gt
        self.T_min = T_min
        self.neighbour = neighbour
        self.energy = energy
        self.accept = accept
        self.cooling = cooling
        
        self.T = T_max
        self.best = init()
        self.step = 0

    def iteration(self):
        next = self.neighbour(self.T, self.best)
        deltaE = self.energy(next) - self.energy(self.best)
        if self.compare(deltaE, 0):
            self.best = next
        elif random.random() < self.accept(self.T, deltaE):
            self.best = next

    def cool(self):
        self.T = self.cooling(self.T, self.best)

    def execute(self):
        while self.T > self.T_min:
            self.iteration()
            self.cool()
            self.log()
            self.step += 1
        print('\nf(' + str(self.best) + ') = ' + str(self.energy(self.best)))
        return self.best
    
    def log(self):
        if(self.step % 100 == 0):
            print(str(self.step) + " T="+str(self.T) +
                  "\tf(" + str(self.best) + ") = " + str(self.energy(self.best)))

# standart configuration parameters and functions for Simulated Annealing
t_min = 1e-5
t_max = 1

def init():
    return np.array([0.1, 1.4])

def neighbour(T, x):
    #return x + random.uniform(-T, T, x.shape)
    return random.normal(x, max(T, 0.1), x.shape)

def accept(T, deltaE, k=0.1):
    return math.exp(-(deltaE)/k/T)

def cooling(T, best):
    return T*0.99

In [37]:
# knapsack problem data set generator

DIM = 60
MAX_VALUE = 100000000
MAX_WEIGHT = 100000000
C = 1000000000
W = np.random.randint(MAX_WEIGHT, size=DIM)
V = np.random.randint(MAX_VALUE, size=DIM)
print(W)
print(V)

[87540576 66065692 49226013 85060943 12799214  9415695 59431308 58039401
 17749648 95841709 86389235 32943633 25451920 16595691 14802301 97944877
 93948632 11427307 57372610 75135754 14955964 64227719 26852132 58896059
 80524126 32466765 18111611 75088581 48960831 70552949 58452909 70076895
 41355657 94231259 36483855 78746993 56311435 91991131 29534310  1379266
 17433326 79958990 41635557 28406442 51816603 66977069 70914858 61934527
 90511996 45198418 31283772 87701575 11873877 64418680 85330416 54162549
 80861745 66779148  1369243 79464001]
[29218712 75762701 49815703 28447810  1897777 94422043  9972063 12749245
 94971555 41415730 13549784 36367465 45708079 26470841 14091578 34939184
  4230864   301552 19088302 88662974 93571175 66644288 61466728  7123415
  7510451 43075173 31940070 69902083  4132943 88554902 13212520 60014839
 94228028 43133848 85524725 84973269 55285591 30633975  8145527  7584374
 76389229 95097678 37545817 23854013 12975533 29961373 91140226 45262695
 94380050 638

In [38]:
from functools import partial


class KnapsackAnnealer:
    
    @staticmethod
    def kp_init(DIM):
        return np.random.randint(2, size=DIM)  # DIM zeros and ones

    
    # naive choice of next knapsack
    @staticmethod
    def kp_neighbour(DIM, cost_fn, T, knapsack):
        new_knapsack = np.copy(knapsack)
        new_knapsack[random.randint(DIM)] ^= 1  # flip the bit
        while cost_fn(new_knapsack) == 0:
            new_knapsack[random.randint(DIM)] ^= 1
        return new_knapsack

    
    @staticmethod
    def kp_cost(W, V, C, knapsack):
        total_weight = sum([W[i] for i, chosen in enumerate(knapsack) if chosen])
        total_value = sum([V[i] for i, chosen in enumerate(knapsack) if chosen])
        return 0 if total_weight > C else total_value

    
    @staticmethod
    def kp_accept(T, deltaE, k=0.1):
        return math.exp(deltaE / k / T)

    
    def __init__(self, weights, values, capacity):
        self.W = weights
        self.V = values
        self.C = capacity
        self.DIM = len(self.W)
        
        init = partial(kp_init, self.DIM)
        self.cost = partial(kp_cost, self.W, self.V, self.C)
        neighbour = partial(kp_neighbour, self.DIM, self.cost)
        
        self.annealer = SimulatedAnnealing(False, t_min, t_max, init,
                                           neighbour, cost, kp_accept, cooling)

        
    def run(self):
        return self.annealer.execute()

In [39]:
# maximization of the 0-1 Knapsack Problem with data taken from
# https://people.sc.fsu.edu/~jburkardt/datasets/knapsack_01/knapsack_01.html

# DIM = 15
# W = [70, 73, 77, 80, 82, 87, 90, 94, 98, 106, 110, 113, 115, 118, 120]
# V = [135, 139, 149, 150, 156, 163, 173, 184, 192, 201, 210, 214, 221, 229, 240]
# C = 750
# OPTIMUM = 1458
# OPTIMAL_KNAPSACK = [1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]

W = [382745, 799601, 909247, 729069, 467902, 44328, 34610, 698150,
     823460, 903959, 853665, 551830, 610856, 670702, 488960, 951111,
     323046, 446298, 931161, 31385, 496951, 264724, 224916, 169684]
V = [825594, 1677009, 1676628, 1523970, 943972, 97426, 69666, 1296457,
     1679693, 1902996, 1844992, 1049289, 1252836, 1319836, 953277,
     2067538, 675367, 853655, 1826027, 65731, 901489, 577243, 466257, 369261]
C = 6404180

annealer = KnapsackAnnealer(W, V, C)
solution = annealer.run()
annealer.cost(solution)

OPTIMUM = 13549094
OPTIMAL_KNAPSACK = [1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1]

mismatches = 0
for i, item in enumerate(solution):
    mismatches += 1 if solution[i] != OPTIMAL_KNAPSACK[i] else 0
print(str(mismatches) + " items in the selection mismatch with the optimal solution")
print("{0:.3f}% less optimal then the most optimal solution".format((1 - annealer.cost(solution)/OPTIMUM) * 100))

0 T=0.99	f([0 1 0 0 0 1 1 0 0 1 1 0 1 1 1 0 0 1 1 0 0 0 0 0]) = 11797720
100 T=0.36237201786049694	f([0 1 0 0 1 1 1 0 0 1 1 1 1 1 0 0 0 0 1 1 0 1 1 0]) = 13093280
200 T=0.13263987810938213	f([1 1 0 0 1 1 1 0 0 1 1 0 1 1 1 0 0 1 0 1 0 1 1 1]) = 13219751
300 T=0.0485504851305729	f([1 1 0 0 1 1 1 0 0 1 1 0 1 1 1 0 0 1 0 1 0 1 1 1]) = 13219751
400 T=0.017771047742294682	f([1 1 0 0 1 1 1 0 0 1 1 0 1 1 1 0 0 1 0 1 0 1 1 1]) = 13219751
500 T=0.006504778211990459	f([1 1 0 0 1 1 1 0 0 1 1 0 1 1 1 0 0 1 0 1 0 1 1 1]) = 13219751
600 T=0.0023809591983979563	f([1 1 0 0 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 1 1]) = 13390040
700 T=0.0008715080698656353	f([1 1 0 0 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 1 1]) = 13390040
800 T=0.00031900013925143135	f([1 1 0 0 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 1 1]) = 13390040
900 T=0.00011676436783668758	f([1 1 0 0 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 1 1]) = 13390040
1000 T=4.2739534936551264e-05	f([1 1 0 0 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 1 1]) = 13390040
1100 T=1.564